<a href="https://colab.research.google.com/github/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/Notebooks/Bases_Funcoes_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook dedicado ao tratamento de dados, criação de funções e classes

Neste notebook iremos importar as bibliotecas necessárias, fazer o tratamento inicial dos dados e criar as funções e classes necessárias para as avaliações do projeto

Se você não leu, recomendo que leia o [Readme](https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/README.md). E se quiser ver as análises para a seleção dos modelos veja este [notebook](https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/Notebooks/Seleciona_Modelo.ipynb).

## Importando as bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, Normalizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, classification_report, ConfusionMatrixDisplay, f1_score
from sklearn.base import BaseEstimator, TransformerMixin
from joblib import dump, load

## Importando e tratando os dados


### Importando a base de dados
A base original está neste desafio do [Kaggle](https://www.kaggle.com/S%C3%ADrio-Libanes/covid19)

In [ ]:
dados = pd.read_excel("https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true")

### Tratando os dados

#### Primeiro passo: criar as bases dados_tratados e dados_LE
Aqui será realizado alguns tratamentos:
* Retirar as linhas com marcação de UTI = 1
* Remarcar a coluna ICU (UTI em inglês), com a informação se o paciente visitante (PATIENT_VISIT_IDENTIFIER) chegou na UTI, independente da janela (campo WINDOW)
* Tratar os dados que não estão disponíveis com base na medição anterior ou posterior
  * Caso ainda sobre valores indisponíveis, as linhas serão excluídas
* Remover as colunas que tem o mesmo valor em todas as linhas da base
* Reiniciar o index para a numeração ficar de 0 até o número de linhas.
* Transformar o campo demográfico 'AGE PERCENTIL' que está em formato de texto

A base dados_LE irá alterar o formato do campo AGE_PERCENTIL para valores numéricos com base na função [LabelEncoder() do scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

Caso a base passe a ter mais de um campo categórico de texto a base dados_LE não será criada e será comunicado o ocorrido.

In [ ]:
print("\nTratamento da Base com Label Encoder")

pacientes_UTI = dados[['PATIENT_VISIT_IDENTIFIER','ICU']].query('ICU == 1').groupby('PATIENT_VISIT_IDENTIFIER').min()
dados_tratados = dados.query('ICU != 1').drop('ICU', axis=1)
dados_tratados = dados_tratados.join(pacientes_UTI, on='PATIENT_VISIT_IDENTIFIER', how='left')
dados_tratados['ICU'] = dados_tratados['ICU'].fillna(0) 
print("\nRemovemos as linhas com ICU(UTI) igual a 1 e remarcamos a coluna com base no PATIENT_VISIT_IDENTIFIER que chegou na UTI")
print(f"Distribuição de ICU na base tratada (%)\n{dados_tratados['ICU'].value_counts(normalize=True)*100}")

features_continuas_colunas = dados_tratados.iloc[:, 13:-2].columns
features_continuas = dados_tratados.groupby("PATIENT_VISIT_IDENTIFIER",as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
features_categoricas = dados_tratados.iloc[:, :13]
saida = dados_tratados.iloc[:, -2:]
dados_tratados = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
dados_tratados.columns = dados.columns
print("\nAjustamos os valores continuos que estavam com Nam para o valor anterior ou posterior")

descricao = dados_tratados.describe().T
colunas_sem_variacao = descricao[descricao['min'] == descricao['max']].index
if len(colunas_sem_variacao) !=0:
  dados_tratados.drop(colunas_sem_variacao, axis=1)
  print("\nRemovemos as colunas que os valores são iguais para todas as linhas")

linhas_com_nam = dados_tratados.describe(include='all').loc['count'].max()-dados_tratados.describe(include='all').loc['count'].min()
if linhas_com_nam !=0:
  if linhas_com_nam <= len(dados_tratados)*.1:
    dados_tratados.dropna(inplace=True)
    print(f"\nAs linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_tratados):.2%} do total) foram eliminadas")
  else:
    print(f"\nTemos linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_tratados):.2%} do total) precisam ser tratadas")

dados_tratados.reset_index(drop=True, inplace=True)
print(f"\nO index foi resetado: {dados_tratados.index}")

colunas_categoricas = list(set(dados_tratados.columns)-set(dados_tratados.describe().columns)-{'WINDOW'})
if len(colunas_categoricas) ==1:
  LE = LabelEncoder()
  LE.fit(np.ravel(dados_tratados[colunas_categoricas]))
  dados_LE = dados_tratados.copy()
  dados_LE[colunas_categoricas] = LE.transform(np.ravel(dados_LE[colunas_categoricas]))
  print(f"\nColuna com objeto categórico ({colunas_categoricas[0]}) foi transformada em numérica no DataFrame dados_LE.\nFormato: {dados_LE.shape}")
else:
  print(f"\nColunas com objetos categóricos precisam ser tratados: {', '.join(colunas_categoricas)}")
  
print(f"\nFormato final do DataFrame dados_tratados: {dados_tratados.shape}\n")


Tratamento da Base com Label Encoder

Removemos as linhas com ICU(UTI) igual a 1 e remarcamos a coluna com base no PATIENT_VISIT_IDENTIFIER que chegou na UTI
Distribuição de ICU na base tratada (%)
0.0    67.375887
1.0    32.624113
Name: ICU, dtype: float64

Ajustamos os valores continuos que estavam com Nam para o valor anterior ou posterior

Removemos as colunas que os valores são iguais para todas as linhas

As linhas ainda com Nam (5.0 linhas, 0.36% do total) foram eliminadas

O index foi resetado: RangeIndex(start=0, stop=1405, step=1)

Coluna com objeto categórico (AGE_PERCENTIL) foi transformada em numérica no DataFrame dados_LE.
Formato: (1405, 231)

Formato final do DataFrame dados_tratados: (1405, 231)



#### Segundo passo: criar a base dados_OHE
A base dados_OHE irá alterar o formato do campo AGE_PERCENTIL para valores numéricos com base na função [OneHotEncoder() do scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html). Este modelo transforma o campo em diversas colunas binárias, uma para cada categoria.

Caso a base passe a ter mais de um campo categórico de texto a base dados_OHE não será criada e será comunicado o ocorrido.

In [ ]:
print("\nTratamento da Base com One Hot Encoder")

if len(colunas_categoricas) ==1:
  categorica = np.array(dados_tratados[colunas_categoricas]).reshape(-1, 1)
  OHE = OneHotEncoder()
  categorica_OHE = pd.DataFrame(OHE.fit_transform(categorica).toarray())
  dados_OHE = pd.concat([dados_tratados.drop(colunas_categoricas, axis=1), categorica_OHE], ignore_index=True, axis=1)
  colunas = list(dados_tratados.columns)
  colunas.remove(colunas_categoricas[0])
  colunas_novas = list(dados_tratados[colunas_categoricas[0]].unique())
  colunas_novas.sort()
  colunas.extend(colunas_novas)
  dados_OHE.columns = colunas
  print(f"\nTrocamos o campo AGE_PERCENTIL pelos campos binários {', '.join(colunas_novas)}")
  print(f"\nFormato final do DataFrame dados_OHE: {dados_OHE.shape}")
else:
  print(f"\nColunas com objetos categóricos precisam ser tratados: {', '.join(colunas_categoricas)}")


Tratamento da Base com One Hot Encoder

Trocamos o campo AGE_PERCENTIL pelos campos binários 10th, 20th, 30th, 40th, 50th, 60th, 70th, 80th, 90th, Above 90th

Formato final do DataFrame dados_OHE: (1405, 240)


## Funções

### adiciona_janelas
Junta a base de duas janelas, permitindo fazer a avaliação de janelas seguintes a inicial, conforme o tempo de permanencia do visitante for aumentando.

In [ ]:
def adiciona_janela(dados_anterior, dados_janela, colunas=features_continuas_colunas):
  """
  ________________________________________________________________________________________________________________
  ENTRADAS
  --------
  dados_anterior: DataFrame
      dados da janela atual que está sendo observada. Por exemplo o filtro WINDOW == '0-2'

  dados_janela: DataFrame
      dados da janela que será adcionada. Seguindo o exemplo o filtro WINDOW == '2-4'

  colunas: list
      lista de colunas que devem ser adiciondos, não deveríamos adicionar novamente os dados categóricos por 
      exemplo. Por isso, por padrão são utilizadas as features continuas
  ________________________________________________________________________________________________________________
  SAIDAS
  ------
  DataFrame
      Nova base de dados partindo do dados_janela e adcionando as colunas seleciondas da base dados_anterior, 
      também adicionando a variação dessas colunas contra a janela anterior e removendo o que não for útil
  """
  d1 = dados_janela.set_index('PATIENT_VISIT_IDENTIFIER')
  d2 = dados_anterior.set_index('PATIENT_VISIT_IDENTIFIER')
  sufixo = ' ' + str(dados_anterior.loc[0,"WINDOW"])
  dados_novo = d1.join(d2[colunas], how='left', rsuffix=sufixo).reset_index()

  for _ in colunas:
    coluna = 'var ' + _ + sufixo
    coluna_ = _ + sufixo
    dados_novo[coluna] = dados_novo[_] - dados_novo[coluna_]

  descricao = dados_novo.describe().T
  colunas_sem_variacao = descricao[descricao['min'] == descricao['max']].index
  if len(colunas_sem_variacao) !=0:
    dados_novo.drop(colunas_sem_variacao, axis=1)
    print("\nRemovemos as colunas que os valores são iguais para todas as linhas")

  linhas_com_nam = dados_novo.describe(include='all').loc['count'].max()-dados_novo.describe(include='all').loc['count'].min()
  if linhas_com_nam !=0:
    if linhas_com_nam <= len(dados_novo)*.1:
      dados_novo.dropna(inplace=True)
      print(f"\nAs linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_novo):.2%} do total) foram eliminadas")
    else:
      print(f"\nTemos linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_novo):.2%} do total) precisam ser tratadas")
  print(f'\nBase nova de tamanho: {dados_novo.shape}')
  return dados_novo

## Classes

### remove_corr
classe utilizada para seleção de dados, removendo os que tem alta correlação entre si e baixa correlação com o y alvo.

In [ ]:
class remove_corr(BaseEstimator, TransformerMixin):
  """
  ________________________________________________________________________________________________________________
  Seleciona os dados com base na correlação deles entre si (dentro do X) e com a variável objetivo (y)
  ________________________________________________________________________________________________________________
  PARAMETROS
  ----------
  corr_maxima: float
      define o máximo valor permitido para as correlações entre as variávies de X, acima disso as variáveris X
      serão desconsideradas (mantendo apenas uma entre as duas que tem alta correlação entre si)

  corr_minima: float
      define o valor minimo para a correlação entre o y e as variaveis de X, abaixo disso as variaves de X 
      serão eliminadas
  ________________________________________________________________________________________________________________
  ATRIBUTOS
  ---------
  excluir: list
      Lista das colunas/variáveris a serem excluidas do X, criado pelo método fit()

  """
  def __init__( self, corr_maxima = 0.95, corr_minima=0.05):
    self.corr_maxima = corr_maxima
    self.corr_minima = corr_minima

  def fit( self, X, y):
    """
    ________________________________________________________________________________________________________________
    faz o fit do modelo, salvando uma lista de colunas/variáveris a serem excluidas do X
    ________________________________________________________________________________________________________________
    ENTRADAS
    ----------
    X: DataFrame
        a base das variáveis utilizadas para tentar definir o y

    y: Series
        a variável objetivo que buscamos prever
    ________________________________________________________________________________________________________________
    SAIDA
    -----
    objeto da classe remove_corr
        modelo de seleção já configurado aguardando a utilização do método transform()
        
    """
    corr = X.corr().abs()
    corr_diagonal = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
    self.excluir = [coluna for coluna in corr_diagonal.columns if any(corr_diagonal[coluna] > self.corr_maxima)]
    X_ = X.drop(self.excluir, axis=1)
    X_ = X_.join(y, how='left')
    X_corr = X_.corr().abs()
    self.excluir.extend(list(X_corr.query('ICU < @self.corr_minima')['ICU'].index))

    return self 
    
  def transform(self, X, y = None):
    """
    ________________________________________________________________________________________________________________
    remove as colunas/variáveis definidas pelo método fit() do DataFrame X passado
    ________________________________________________________________________________________________________________
    ENTRADAS
    ----------
    X: DataFrame
        a base das variáveis utilizadas para tentar encontrar definir o y que será transformada

    y: Series
        a variável objetivo que buscamos prever. 
        Não será utilizada e não precisa ser passada.
    ________________________________________________________________________________________________________________
    SAIDA
    -----
    DataFrame
        novo DataFrame do X após a exclusão das colunas/variáveis definidas no fit()
        
    """
    X = X.drop(self.excluir, axis=1)
    return X

### seleciona_modelo
classe para testar as melhores configurações e criar um pipeline dado um modelo de classificação definido

In [ ]:
class seleciona_modelo():
  """
  ________________________________________________________________________________________________________________
  A partir de um modelo de classificação definido, está classe permite você testar diversas opções de pipelines
  e configurações do modelo escolhido, com o objetivo de encontrar o que possuí maior precisão para definir o y.

  Além disso, após os testes podemos criar o melhor pipeline, treina-lo, testa-lo e salva-lo em um aquivo
  ________________________________________________________________________________________________________________
  PARAMETROS
  ----------
  modelo: modelo de classificação do sklearn
      modelo que será utilizado nos testes do pipeline e de seus paramêtros, bem como no pipeline final
  ________________________________________________________________________________________________________________
  ATRIBUTOS
  ---------
  metrica: str
      metrica utilizada nos cálculos comparativos dos testes, definida no método testa_parametros()
  
  Xs: dict
      dicionários dos Xs testados com base nas bases envidas no método testa_parametros()

  ys: dict
      dicionários dos ys testados com base nas bases envidas no método testa_parametros()

  resultados: DataFrame
      DataFrame com os resultados do método testa_parametro(), tem os resultados e as etapas do pipeline utilizado

  indice: int
      indice da linha dos resultados que iremos utilizar para criar o pipeline final, criado no método cria_pipeline()

  pipe: Pipeline
      pipeline final criado no método cria_pipeline()

  base: str
      Nome da base que teve o melhor resultado, para utilização na avaliação das métricas do pipeline final.
      Criada no método cria_pipeline()
  
  modelo_final: Pipeline
      modelo do pipeline utilizado para ser salvo no arquivo. 
      Criado no método cria_modelo_final() ou cria_salva_modelo_final()
  """
  def __init__( self, modelo):
    self.modelo = modelo

  def testa_parametros(self, parametros, lista_dados=[(dados_LE, 'LE')], lista_reescalas=['nenhuma'], 
                      selecao_menor=-30, selecao_maior=30, passo=30, n_splits = 5, n_repeats=10, metrica='roc_auc'):
    """
    ________________________________________________________________________________________________________________
    testa deiveros parametros, seleções e reescalas conforme solicitação do usuário

    por padrão todas os testes colocam no lógica do pipeline um objeto da classe remove_corr(), devido a testes
    anteriores demonstrarem a eficácia da utilização desta seleção prévia
    ________________________________________________________________________________________________________________
    ENTRADAS
    --------
    parametros: dict
        dicionário com os parametros do modelo que queremos testar. lembrando que o formato de cada item é o nome 
        parametro e a lista de valores que se deseja testar

    lista_dados: list de tuplas
        lista contendo tuplas com a base que queremos testar (DataFrame) e nome que iremos identifica-la (str)
    
    lista_reescalas: list
        lista deve conter as funções de reescala que queremos utilizar ou a string 'nenhuma'
    
    selecao_menor: int
        valor do percentual (será divido por 100) que queremos distanciar da média no paramêtro threshold da função
        SelectFromModel. Este é o valor inferior do range que será testado.
        Os valores menores ou iguais a -100 do range farão com que não se aplique essa segunda seleção no número
    
    selecao_maior: int
        valor do percentual (será divido por 100) que queremos distanciar da média no paramêtro threshold da função
        SelectFromModel. Este é o valor superio do range que será testado.
        Os valores menores ou iguais a -100 do range farão com que não se aplique essa segunda seleção no número

    passo: int
        valor do aumento gradual que será colocado na selecao_menor até chegar na selecao_maior dentro do range.
    
    n_splits: int
        número de divisões que será realizada no objeto RepeatedStratifiedKFold durante os testes

    n_repeats: int
        número de repetições que será realizada no objeto RepeatedStratifiedKFold durante os testes
    
    metrica: str
        metrica utilizada nos cálculos comparativos dos testes
    ________________________________________________________________________________________________________________
    SAIDA
    -----
    DataFrame
        Os resultados tratados já selecionando os 5 melhores, do pior para o melhor, utilizando como métrica o 
        valor do limite inferior do intervalo de confiança de 95%
    """
    self.metrica = metrica  
    cv = RepeatedStratifiedKFold(n_splits = n_splits, n_repeats = n_repeats)
    grid = GridSearchCV(self.modelo,parametros, scoring=metrica,cv=cv)
    melhores = {'base':[],'reescala':[],'selecionador':[],'modelo':[],'parametros':[],self.metrica:[],'desvio_padrao':[]}
    self.Xs = {}
    self.ys = {}
    for dados, base in lista_dados:
      dados = dados.sample(frac=1).reset_index(drop=True)
      y = dados["ICU"]
      X = dados.drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
      self.Xs = {base: X}
      self.ys = {base: y}
      _ = remove_corr().fit(X,y)
      X = _.transform(X)
      modelo_ = self.modelo.fit(X,y)
      for i in range(selecao_menor,selecao_maior+1,passo):
        if i > -100:
          threshold_i = str(1+i/100)+'*mean'
          seletor = SelectFromModel(modelo_, threshold=threshold_i).fit(X,y)
          X_selecionado = seletor.transform(X)
        else:
          threshold_i = 'sem seleção adicional'
          seletor = {'threshold':threshold_i,'colunas':X.columns}
          X_selecionado = X.copy()
        for item in lista_reescalas:
          if item != 'nenhuma':
            X_revisto = item.fit_transform(X_selecionado)
          else:
            X_revisto = X_selecionado
          resultados= grid.fit(X_revisto,y)
          desvio = resultados.cv_results_['std_test_score'][resultados.best_index_]
          melhores['base'].append(base)
          melhores['reescala'].append(item)
          melhores['selecionador'].append(seletor)
          melhores['modelo'].append(resultados)
          melhores['parametros'].append(resultados.best_params_)
          melhores[self.metrica].append(resultados.best_score_)
          melhores['desvio_padrao'].append(desvio)
          print(f'Finalizada iteração: seleção - {threshold_i}, reescala - {item}, base - {base}')

    self.resultados = pd.DataFrame(melhores)
    return self.classifica_resultados().tail()

  def classifica_resultados(self, ordenar='inferior'): 
    """
    ________________________________________________________________________________________________________________
    ajusta os resultados num formato mais amigável e já ordena o campo informado, do menor para o maior
    ________________________________________________________________________________________________________________
    ENTRADA
    -------
    ordenar: str
        campo que desejamos ordenar no fim do tratamento
    ________________________________________________________________________________________________________________
    SAIDA
    -----
    DataFrame
        Os resultados tratados ordenados pelo campo definido em ordenar, do menor para o maior
    """
    r = self.resultados.copy()
    r['inferior']=r[self.metrica]-2*r['desvio_padrao']
    r['superior']=r[self.metrica]+2*r['desvio_padrao']
    colunas = r.columns.tolist()
    colunas = colunas[2:]+colunas[:2]
    r = r[colunas]
    r['threshold']=""
    r['qtd_variaveis']=""
    for i in r.index.values:
      if type(r.loc[i,'selecionador']) == dict:
        r.loc[i,'threshold'] = r.loc[i,'selecionador']['threshold']
        r.loc[i,'qtd_variaveis'] = len(r.loc[i,'selecionador']['colunas'])
      else:
        r.loc[i,'threshold'] = r.loc[i,'selecionador'].threshold
        r.loc[i,'qtd_variaveis'] = len(r.loc[i,'selecionador'].get_feature_names_out())
    for titulo in r.loc[0,'parametros'].keys():
      r[titulo]=''
      for i in r.index.values:
        r.loc[i,titulo] = r.loc[i,'parametros'][titulo]
    r.drop(['selecionador','modelo','parametros'], axis=1, inplace=True)
    return r.sort_values(ordenar)

  def cria_pipeline(self, indice=None):
    """
    ________________________________________________________________________________________________________________
    cria um pipeline com base nos resultados obtidos que fica salvo no atributo 'pipe'
    ________________________________________________________________________________________________________________
    ENTRADA
    -------
    indice: str
        indice da base de resultados que queremos utilizar na criação do pipeline. Se não for passado irá
        utilizar o indice do maior limite inferior do intervalor de confiança de 95%.
    """
    pipe = [('seletor1',remove_corr())]
    
    if indice:
      self.indice = indice
    else:
      _ = self.classifica_resultados().reset_index().iloc[-1]
      self.indice = _['index']
    
    if type(self.resultados.loc[self.indice,'selecionador']) != dict:
      pipe.append(('seletor2',self.resultados.loc[self.indice,'selecionador']))
    if self.resultados.loc[self.indice,'reescala'] != 'nenhuma':
      pipe.append(('reescala',self.resultados.loc[self.indice,'reescala']))
    pipe.append(('modelo',self.resultados.loc[self.indice,'modelo'].best_estimator_))
    self.pipe = Pipeline(pipe)
    self.base = self.resultados.loc[self.indice,'base']
    return None

  def roda_pipeline_metricas(self, random_state=None):
    """
    ________________________________________________________________________________________________________________
    com base no pipeline criado, plota a matriz de confusão, os reportes de classificação e o ROC AUC com base
    em um split entre treino e dados aleatório da base definida como melhor 
    ________________________________________________________________________________________________________________
    ENTRADA
    -------
    random_state: int
        caso se queira definir uma semente para que se tenha o mesmo resultado ao se rodar novamente este método
    """
    X_treino, X_teste, y_treino, y_teste = train_test_split(self.Xs[self.base], self.ys[self.base], 
                                                            stratify=self.ys[self.base], random_state=random_state)
    self.pipe.fit(X_treino, y_treino)

    ConfusionMatrixDisplay.from_predictions(y_teste, self.pipe.predict(X_teste))
    plt.show()
    print('\n', classification_report(y_teste, self.pipe.predict(X_teste)))
    print(f'ROC AUC: {roc_auc_score(y_teste, self.pipe.predict_proba(X_teste)[:,1]):.2%}')
    
    return None
 
  def cria_modelo_final(self):
    """
    ________________________________________________________________________________________________________________
    salva o pipeline numa atibuto modelo_final que pode ser preservado para outras comparações ou ser salvo em um 
    arquivo
    """
    self.modelo_final = self.pipe.fit(self.Xs[self.base], self.ys[self.base])
    return self.modelo_final

  def salva_modelo_final(self, nome_arquivo='modelo_final.joblib'):
    """
    ________________________________________________________________________________________________________________
    salva o modelo final em um arquivo joblib para utilização posterior ou para utilização na produção
    ________________________________________________________________________________________________________________
    ENTRADA
    --------
    nome_arquivo: str
        nome do arquivo onde ficará salvo o modelo
    ________________________________________________________________________________________________________________
    SAIDA
    -----
    str
        informa que o modelo foi salvo e o nome do arquivo
    """
    dump(self.modelo_final, nome_arquivo)
    return 'modelo salvo como: ' + nome_arquivo

  def cria_salva_modelo_final(self, nome_arquivo='modelo_final.joblib'):
    """
    ________________________________________________________________________________________________________________
    salva o pipeline numa atibuto modelo_final que pode ser preservado para outras comparações. Também salva o 
    modelo final em um arquivo joblib para utilização posterior ou para utilização na produção
    ________________________________________________________________________________________________________________
    ENTRADA
    --------
    nome_arquivo: str
        nome do arquivo onde ficará salvo o modelo
    ________________________________________________________________________________________________________________
    SAIDA
    -----
    str
        informa que o modelo foi salvo e o nome do arquivo
    """
    self.modelo_final = self.pipe.fit(self.Xs[self.base], self.ys[self.base])
    dump(self.modelo_final, nome_arquivo)
    return 'modelo salvo como: ' + nome_arquivo